In [13]:
import os
import pandas as pd
import numpy as np
import openai
import requests
import pandas as pd
import openai
import json

import logging
import time

from typing import List, Dict, Any, Optional
from enum import Enum
from dataclasses import dataclass

import openrouter



In [2]:
# Import the openrouter module

# Store API key in a separate file
with open('../config/openrouter_api_key.txt', 'r') as f:
    api_key = f.read().strip()

openrouter.api_key = api_key

In [3]:
# Store API key in a separate file
with open('../config/api_key.txt', 'r') as f:
    api_key = f.read().strip()

openai.api_key = api_key

In [16]:
# disease = 'Parkinson'

In [6]:
# Define file path
file_path_parquet = os.path.join('../data/filtered', '02_simple_filter.parquet')

# Load the DataFrame from the Parquet file
filtered_df_covariates = pd.read_parquet(file_path_parquet)

# Display the DataFrame
print(filtered_df_covariates)

In [18]:
print(filtered_df_covariates.columns)

Index(['Index', 'NCTId', 'LeadSponsorClass', 'LeadSponsorName', 'Condition',
       'OfficialTitle', 'BriefTitle', 'Acronym', 'StudyType',
       'InterventionType', 'InterventionName', 'InterventionOtherName',
       'InterventionDescription', 'Phase', 'StudyFirstSubmitDate',
       'LastUpdateSubmitDate', 'CompletionDate', 'OverallStatus',
       'BriefSummary', 'IsFDARegulatedDevice', 'StartDate',
       'DetailedDescription', 'ConditionMeshTerm', 'PrimaryOutcomeDescription',
       'SecondaryOutcomeDescription', 'EnrollmentCount', 'EnrollmentType',
       'BaselineCategoryTitle', 'BaselinePopulationDescription',
       'BaselineTypeUnitsAnalyzed', 'OtherOutcomeDescription',
       'EligibilityCriteria', 'StudyPopulation', 'HealthyVolunteers',
       'ReferencePMID', 'LocationCountry', 'PrimaryOutcomeTimeFrame',
       'BaselineMeasureTitle', 'BaselineMeasureUnitOfMeasure',
       'BaselineMeasurementValue', 'GPT_summary'],
      dtype='object')


In [19]:
#test openai    



# response = openai.chat.completions.create(
#     model='gpt-3.5-turbo',
#     messages=[
#         {"role": "user", "content": "Hello, how are you?"}
#     ],
#     max_tokens=50,
#     temperature=0.5,
# )

# print(response.choices[0].message.content.strip())

In [20]:
# import requests
# import json

# # Configuration Variables
# API_KEY = openrouter.api_key       # Ensure this is defined
# SITE_URL = 'https://pd-research.com'          # Your site's URL
# APP_NAME = 'PD Research'                      # Your application's name

# def send_test_prompt():
#     print("Starting the API request...")  # Debugging statement
#     try:
#         # Prepare the payload
#         payload = {
#             "model": "openai/o1-mini-2024-09-12",  # Updated model
#             "messages": [
#                 {
#                     "role": "user",
#                     "content": "What is the average age Parkinson's disease starts?"
#                 }
#             ],
#             "max_tokens": 150,  # Increased max_tokens
#             "temperature": 0.7,  # Increased temperature for more varied responses
#         }

#         # Make the POST request to the OpenRouter API
#         response = requests.post(
#             url="https://openrouter.ai/api/v1/chat/completions",
#             headers={
#                 "Authorization": f"Bearer {API_KEY}",
#                 "HTTP-Referer": SITE_URL,  # Optional: Include if you want your app featured
#                 "X-Title": APP_NAME,        # Optional: Shows your app name on OpenRouter rankings
#             },
#             data=json.dumps(payload)
#         )

#         print("API request sent. Awaiting response...")  # Debugging statement

#         # Check if the request was successful
#         if response.status_code == 200:
#             print("Received successful response from the API.")  # Debugging statement
#             response_data = response.json()

#             # Print the full JSON response for debugging
#             print("Full Response JSON:")
#             print(json.dumps(response_data, indent=4))

#             # Extract the assistant's reply
#             try:
#                 assistant_reply = response_data['choices'][0]['message']['content'].strip()
#                 print(f"Assistant's reply: {assistant_reply}")
#             except (KeyError, IndexError) as e:
#                 print("Error extracting assistant's reply:", e)
#                 print("Full response:", json.dumps(response_data, indent=4))
#         else:
#             print(f"Request failed with status code {response.status_code}")
#             print("Response:", response.text)

#     except requests.exceptions.RequestException as e:
#         print("An error occurred while making the request:", e)

# # --- Execute the Test Prompt ---
# send_test_prompt()

In [21]:
def generate_prompt(self, context: str) -> str:
    """Generate analysis prompt from context"""
    return f"""
Analyze whether brain stimulation was used in this trial. If so, provide details.

IMPORTANT: Respond ONLY with a JSON object in the EXACT format below. Do not include any additional text or explanations.

{{
    "brain_stimulation_used": "Yes" or "No",
    "stimulation_details": {{
        "primary_type": "e.g., tDCS, TMS, tACS, DBS, etc." or null,
        "is_noninvasive": true or false,
        "primary_target": "Primary brain region or null",
        "secondary_targets": ["List of secondary regions"] or [],
        "stimulation_parameters": {{
            "intensity": "e.g., 2mA" or null,
            "duration": "e.g., 20 minutes" or null
        }}
    }},
    "confidence_level": "High", "Medium", or "Low",
    "relevant_quotes": ["Direct quotes supporting the analysis"]
}}

Context:
{context}
"""


In [22]:
import openai
import os
import time

from autogen import ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.capabilities.teachability import Teachability

###############################################################################
# 1) Hardcode your OpenRouter API key and base URL
###############################################################################
openrouter_api_key = "sk-or-v1-REPLACE_ME_WITH_REAL_KEY"
openai.api_key = openrouter_api_key
openai.api_base = "https://openrouter.ai/api/v1"

# Optionally set extra OpenRouter headers
openai.request_headers = {
    "Authorization": f"Bearer {openrouter_api_key}",
    "X-Title": "MyMultiAgentApp",
    "HTTP-Referer": "https://example.com",
}

###############################################################################
# 2) Define multiple model names that are known to work with your key
#    (If you don't have GPT-4 access, remove "gpt-4".)
###############################################################################
model_list = [
    "gpt-4",
    "anthropic/claude-3.5-sonnet",
    "meta-llama/llama-3-70b-instruct",
]

###############################################################################
# 3) Build an agent for each model, with Teachability
###############################################################################
def build_agent_for_model(model_name: str):
    """
    Creates a ConversableAgent configured for a specific model.
    Attaches Teachability with a local vector DB folder 
    unique to the model_name.
    """
    config_list = [
        {
            "model": model_name,
            "temperature": 0.5,   # or any you prefer
            "max_tokens": 4096,   # might need a bigger limit for some models
            "top_p": 1.0,
        }
    ]

    agent = ConversableAgent(
        name=f"agent_{model_name}",
        llm_config={
            "config_list": config_list,
            "timeout": 120,
            "cache_seed": None,
        },
    )

    # Use a unique DB path for each model
    db_path = f"./tmp/db_{model_name.replace('/','_').replace('-','_')}"

    teachability = Teachability(
        verbosity=1,
        reset_db=True,       # Clears any prior memos each run
        path_to_db_dir=db_path,
        recall_threshold=1.3,
    )
    teachability.add_to_agent(agent)

    return agent

###############################################################################
# 4) Main Demo
###############################################################################
def main():
    # Hardcoded sports betting text for "teaching"
    sports_betting_text = """\
A spread bet in sports betting is a wager based on the margin of victory in a game,
commonly used in popular sports like football and basketball. When you bet on the spread,
you're predicting how much a team will win or lose by.

If you bet on the favorite (shown with a '-' sign), that team must win by more than the
specified point spread for your bet to succeed. If you bet on the underdog (shown with a '+' sign),
they can lose by fewer points than the spread — or win outright — for your bet to win.

The spread helps balance the odds between teams, making the bet more about predicting 
the margin rather than just picking a winner.

Example: If the Bears are -6.5 points against the Packers, the Bears must win by more than 6.5 
points for a Bears -6.5 bet to cash. If the game ends with the Bears winning by exactly 6 points, 
that bet loses. If the underdog (Packers) loses by fewer than 6.5, an underdog bet would win.
"""

    # Create one user proxy. We'll call user.initiate_chat(...) ourselves.
    user = UserProxyAgent(
        name="user",
        human_input_mode="NEVER",   # We handle input() in Python code
        max_consecutive_auto_reply=0,
    )

    # Build an agent for each model
    agents = {}
    for m in model_list:
        agents[m] = build_agent_for_model(m)

    # 4a) Teach each agent the sports-betting text once
    for model_name, agent in agents.items():
        print(f"\n>>> Teaching {model_name} the sports-betting text...\n")
        teach_resp = user.initiate_chat(agent, message=sports_betting_text, clear_history=True)
        print(f"{model_name} replied:", teach_resp.content)

    # 4b) Now we do a Q&A loop: user types question, all agents respond
    print("\n=== Multi-Model Q&A ===")
    print("Type 'exit' to quit.\n")
    while True:
        q = input("You: ").strip()
        if q.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break

        for model_name, agent in agents.items():
            # Start a new chat to confirm it can recall the taught text
            # Or you can do clear_history=False so it accumulates Q&A
            answer = user.initiate_chat(agent, message=q, clear_history=True)
            print(f"\n[{model_name} responds]\n{answer.content}\n")

            # Optional: small delay if you want to separate them visually
            time.sleep(1)

if __name__ == "__main__":
    main()

2024-12-26 11:06:41 - INFO - Sent request to gpt-4
2024-12-26 11:06:41 - INFO - Sent request to gpt-4
2024-12-26 11:06:41 - INFO - Sent request to gpt-4
2024-12-26 11:06:42 - INFO - Sent request to gpt-4
2024-12-26 11:06:42 - INFO - Sent request to gpt-4
2024-12-26 11:06:43 - INFO - Sent request to gpt-4
2024-12-26 11:06:44 - INFO - Sent request to gpt-4
2024-12-26 11:06:44 - INFO - Sent request to gpt-4
2024-12-26 11:06:44 - INFO - Sent request to anthropic/claude-3.5-sonnet
2024-12-26 11:06:46 - INFO - Sent request to gpt-4
2024-12-26 11:06:46 - INFO - Sent request to anthropic/claude-3.5-sonnet
2024-12-26 11:06:47 - INFO - Sent request to anthropic/claude-3.5-sonnet
2024-12-26 11:06:48 - INFO - Sent request to meta-llama/llama-3-70b-instruct
2024-12-26 11:06:48 - INFO - Sent request to anthropic/claude-3.5-sonnet
2024-12-26 11:06:48 - INFO - Sent request to anthropic/claude-3.5-sonnet
2024-12-26 11:06:48 - INFO - Sent request to anthropic/claude-3.5-sonnet
2024-12-26 11:06:48 - INFO


=== Updated LLM Responses ===
    Index        NCTId LeadSponsorClass  \
150  None  NCT06145776            OTHER   
200  None  NCT04174573            OTHER   
232  None  NCT03191916            OTHER   
284  None  NCT02503930        OTHER_GOV   
305  None  NCT02315781            OTHER   

                                       LeadSponsorName  \
150                      Federal University of Paraíba   
200  Maharishi Markendeswar University (Deemed to b...   
232                                     Sanford Health   
284                   Tel-Aviv Sourasky Medical Center   
305                Pacific Parkinson's Research Centre   

                                             Condition  \
150  Parkinson Disease, Cognition Disorder, EEG Wit...   
200                                  Parkinson Disease   
232                                  Parkinson Disease   
284                                  Parkinson Disease   
305                            Depression, Parkinson's   

            

In [23]:
import requests



resp = requests.post('https://textbelt.com/text', {

  'phone': '9163802941',

  'message': 'PD Pipe Line Part 3 Done',

  'key': '138adc496234ca311154757db147f552afa8ba83FfrCKJ36kTJNXq65nlsvvF4Pu',

})

print(resp.json())

{'success': True, 'textId': '10931735240158025', 'quotaRemaining': 594}


In [24]:


1=2


SyntaxError: cannot assign to literal here. Maybe you meant '==' instead of '='? (4266335387.py, line 1)

In [15]:
# old working code 

In [16]:
import os
import requests
import json
import logging
import pandas as pd
import time
from typing import Dict, List, Optional, Union
from dataclasses import dataclass
from enum import Enum

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

class APIError(Exception):
    """Custom exception for API-related errors"""
    pass

class ValidationError(Exception):
    """Custom exception for response validation errors"""
    pass

class BrainStimStatus(str, Enum):
    YES = "Yes"
    NO = "No"

@dataclass
class StimulationParameters:
    frequency: Optional[str] = None
    intensity: Optional[str] = None
    duration: Optional[str] = None

@dataclass
class StimulationDetails:
    primary_type: Optional[str] = None
    is_noninvasive: Optional[bool] = None
    primary_target: Optional[str] = None
    secondary_targets: List[str] = None
    stimulation_parameters: StimulationParameters = None

    def __post_init__(self):
        if self.secondary_targets is None:
            self.secondary_targets = []
        if self.stimulation_parameters is None:
            self.stimulation_parameters = StimulationParameters()

@dataclass
class BrainStimResponse:
    brain_stimulation_used: BrainStimStatus
    stimulation_details: StimulationDetails
    confidence_level: str
    relevant_quotes: List[str]

class BrainStimAnalyzer:
    def __init__(self, api_key: str, site_url: str, app_name: str):
        self.api_key = api_key
        self.site_url = site_url
        self.app_name = app_name
        self.models = {
            "gpt-4": {
                "requires_image": False,
                "max_tokens": 4096,
                "temperature": 0.5,
                "instructions": "Return ONLY a JSON object with no additional explanatory text."
            },
            "anthropic/claude-3.5-sonnet": {
                "requires_image": False,
                "max_tokens": 10000,
                "temperature": 0.5,
                "instructions": "Return ONLY a JSON object with no additional explanatory text."
            },
            "meta-llama/llama-3-70b-instruct": {
                "requires_image": False,
                "max_tokens": 4096,
                "temperature": 0.5,
                "instructions": "Return ONLY a JSON object with no additional explanatory text."
            }
        }

    def extract_json_from_text(self, text: str) -> Optional[str]:
        """Extract JSON object from text that may contain additional content"""
        try:
            # Look for JSON-like content between curly braces
            start = text.find('{')
            end = text.rfind('}')
            if start != -1 and end != -1:
                json_str = text[start:end+1]
                # Validate it's proper JSON
                json.loads(json_str)
                return json_str
        except json.JSONDecodeError:
            pass
        return None

    def validate_brain_stim_response(self, response_json: Dict) -> BrainStimResponse:
        """Validate response structure and content"""
        required_fields = [
            "brain_stimulation_used",
            "stimulation_details",
            "confidence_level",
            "relevant_quotes"
        ]

        # Check required fields
        for field in required_fields:
            if field not in response_json:
                raise ValidationError(f"Missing required field: {field}")

        # Validate brain_stimulation_used
        if not isinstance(response_json["brain_stimulation_used"], str):
            raise ValidationError("brain_stimulation_used must be string")
        if response_json["brain_stimulation_used"] not in ["Yes", "No"]:
            raise ValidationError("brain_stimulation_used must be 'Yes' or 'No'")

        # Convert to BrainStimResponse object
        stim_params = StimulationParameters(**response_json["stimulation_details"]["stimulation_parameters"])
        stim_details = StimulationDetails(
            primary_type=response_json["stimulation_details"]["primary_type"],
            is_noninvasive=response_json["stimulation_details"]["is_noninvasive"],
            primary_target=response_json["stimulation_details"]["primary_target"],
            secondary_targets=response_json["stimulation_details"]["secondary_targets"],
            stimulation_parameters=stim_params
        )

        return BrainStimResponse(
            brain_stimulation_used=BrainStimStatus(response_json["brain_stimulation_used"]),
            stimulation_details=stim_details,
            confidence_level=response_json["confidence_level"],
            relevant_quotes=response_json["relevant_quotes"]
        )

    def send_request(self, model_name: str, prompt: str, max_retries: int = 3) -> BrainStimResponse:
        """Send request to API with retry logic"""
        if model_name not in self.models:
            raise ValueError(f"Unknown model: {model_name}")

        model_config = self.models[model_name]
        prompt = f"{model_config['instructions']}\n{prompt}"

        retries = 0
        backoff = 2  # seconds

        while retries < max_retries:
            try:
                response = requests.post(
                    url="https://openrouter.ai/api/v1/chat/completions",
                    headers={
                        "Authorization": f"Bearer {self.api_key}",
                        "HTTP-Referer": self.site_url,
                        "X-Title": self.app_name,
                        "Content-Type": "application/json"
                    },
                    json={
                        "model": model_name,
                        "messages": [{"role": "user", "content": prompt}],
                        "temperature": model_config["temperature"],
                        "max_tokens": model_config["max_tokens"]
                    },
                    timeout=60
                )

                logging.info(f"Sent request to {model_name}")
                logging.debug(f"Raw response: {response.text}")

                if response.status_code == 200:
                    try:
                        response_data = response.json()
                        # Check if 'choices' and necessary keys exist in the response
                        if 'choices' in response_data and len(response_data['choices']) > 0:
                            choice = response_data['choices'][0]
                            if 'message' in choice and 'content' in choice['message']:
                                content = choice['message']['content'].strip()
                                
                                # Try to parse as JSON directly first
                                try:
                                    response_json = json.loads(content)
                                except json.JSONDecodeError:
                                    # If direct parsing fails, try to extract JSON from text
                                    json_str = self.extract_json_from_text(content)
                                    if not json_str:
                                        raise ValidationError(f"Could not extract valid JSON from response: {content}")
                                    response_json = json.loads(json_str)
                                
                                # Validate response structure and content
                                return self.validate_brain_stim_response(response_json)
                            else:
                                raise ValidationError("Response does not contain 'message' or 'content' keys.")
                        else:
                            raise ValidationError("Response does not contain 'choices' or it's empty.")

                    except (KeyError, IndexError) as e:
                        logging.error(f"Error extracting response content: {e}")
                        raise ValidationError(f"Invalid response structure from {model_name}")

                elif response.status_code in [429, 500, 502, 503, 504]:
                    if retries == max_retries - 1:
                        raise APIError(f"Max retries reached for {model_name}")
                    retries += 1
                    time.sleep(backoff)
                    backoff *= 2
                    continue

                else:
                    raise APIError(f"Request failed with status {response.status_code}: {response.text}")

            except requests.exceptions.RequestException as e:
                if retries == max_retries - 1:
                    raise APIError(f"Request failed after {max_retries} retries: {str(e)}")
                retries += 1
                time.sleep(backoff)
                backoff *= 2
                continue

        raise APIError(f"Failed to get valid response from {model_name}")

class BrainStimAnalysis:
    def __init__(self, api_key: str, site_url: str, app_name: str):
        self.analyzer = BrainStimAnalyzer(api_key, site_url, app_name)

    def process_trial(self, df: pd.DataFrame, output_dir: str = '03_data_ai'):
        """Process trial data and save results"""
        os.makedirs(output_dir, exist_ok=True)
        result_df = df.copy()

        # Initialize response arrays
        responses = []
        
        # Process each row
        for idx, row in df.iterrows():
            context = row.get('DetailedDescription', row.get('BriefSummary', ''))
            prompt = self.generate_prompt(context)

            # Get responses from each model
            for model_name in self.analyzer.models.keys():
                try:
                    response = self.analyzer.send_request(model_name, prompt)
                    responses.append({
                        "Model": model_name,
                        "Response": response
                    })
                    
                    # Update DataFrame with response data
                    self.update_dataframe(result_df, idx, model_name, response)
                
                except (APIError, ValidationError) as e:
                    logging.error(f"Error processing row {idx} with model {model_name}: {str(e)}")
                    continue

        # Save results
        self.save_results(result_df, output_dir)
        return result_df, responses

    def generate_prompt(self, context: str) -> str:
        """Generate analysis prompt from context"""
        return f"""
        Analyze whether brain stimulation was used in this trial. If so, provide details.

        IMPORTANT: Respond ONLY with a JSON object in the EXACT format below. Do not include any additional text or explanations.

        {{
            "brain_stimulation_used": "Yes" or "No",
            "stimulation_details": {{
                "primary_type": "e.g., tDCS, TMS, tACS, DBS, etc." or null,
                "is_noninvasive": true or false,
                "primary_target": "Primary brain region or null",
                "secondary_targets": ["List of secondary regions"] or [],
                "stimulation_parameters": {{
                    "frequency": "e.g., 10Hz" or null,
                    "intensity": "e.g., 2mA" or null,
                    "duration": "e.g., 20 minutes" or null
                }}
            }},
            "confidence_level": "High", "Medium", or "Low",
            "relevant_quotes": ["Direct quotes supporting the analysis"]
        }}

        Context:
        {context}
        """

    def update_dataframe(self, df: pd.DataFrame, idx: int, model_name: str, response: BrainStimResponse):
        """Update DataFrame with response data"""
        suffix = model_name.replace("/", "_").replace("-", "_")
        
        # Update brain stimulation status
        df.at[idx, f"brain_stimulation_used_{suffix}"] = response.brain_stimulation_used.value
        
        # Update stimulation details
        if response.stimulation_details:
            df.at[idx, f"stimulation_details_primary_type_{suffix}"] = response.stimulation_details.primary_type
            df.at[idx, f"stimulation_details_is_noninvasive_{suffix}"] = response.stimulation_details.is_noninvasive
            df.at[idx, f"stimulation_details_primary_target_{suffix}"] = response.stimulation_details.primary_target
            df.at[idx, f"stimulation_details_secondary_targets_{suffix}"] = json.dumps(response.stimulation_details.secondary_targets)
            
            # Update parameters
            params = response.stimulation_details.stimulation_parameters
            df.at[idx, f"stimulation_details_parameters_frequency_{suffix}"] = params.frequency
            df.at[idx, f"stimulation_details_parameters_intensity_{suffix}"] = params.intensity
            df.at[idx, f"stimulation_details_parameters_duration_{suffix}"] = params.duration
        
        # Update confidence and quotes
        df.at[idx, f"confidence_level_{suffix}"] = response.confidence_level
        df.at[idx, f"relevant_quotes_{suffix}"] = json.dumps(response.relevant_quotes)

    def save_results(self, df: pd.DataFrame, output_dir: str):
        """Save results to files"""
        df.to_excel(f'{output_dir}/part_3_covariates_done.xlsx', index=False)
        df.to_parquet(f'{output_dir}/part_3_covariates_done.parquet', index=False)

def main():
    # Configuration
    api_key = openrouter.api_key 
    site_url = 'https://pd-research.com'
    app_name = 'PD Research'
    
    if not api_key:
        logging.error("API_KEY is not set. Please set the OPENROUTER_API_KEY environment variable.")
        exit(1)

    try:
        # Load your DataFrame (replace with actual loading logic)
        df = filtered_df_covariates ## Update with your file path
        
        # Initialize and run analysis
        analysis = BrainStimAnalysis(api_key, site_url, app_name)
        result_df, responses = analysis.process_trial(df)
        
  
        # Save result_df and responses to Excel and Parquet formats
        output_dir = '03_data_ai'
        os.makedirs(output_dir, exist_ok=True)

        # Save result_df
        result_df.to_excel(f'{output_dir}/part_3_covariates_done_v2.xlsx', index=False)
        result_df.to_parquet(f'{output_dir}/part_3_covariates_done_v2.parquet', index=False)

        # Convert responses to DataFrame and save
        responses_df = pd.DataFrame(responses)
        responses_df.to_excel(f'{output_dir}/responses_done_v2.xlsx', index=False)
        responses_df.to_parquet(f'{output_dir}/responses_done_v2.parquet', index=False)

        logging.info(f"Results saved to {output_dir}/part_3_covariates_done_v2.xlsx and {output_dir}/responses_done_v2.xlsx")
        


        # Display results
        print("\n=== Updated LLM Responses ===")
        print(result_df)
        
        # Optional: Display using PrettyTable
        try:
            from prettytable import PrettyTable
            table = PrettyTable()
            table.field_names = ["Model", "Response"]
            for resp in responses:
                table.add_row([resp["Model"], resp["Response"]])
            print("\n=== LLM Responses (PrettyTable) ===")
            print(table)
        except ImportError:
            logging.warning("PrettyTable not installed. Skipping table formatting.")
        
    except Exception as e:
        logging.error(f"Error in main execution: {str(e)}")
        raise




# first_five.to_excel('filtered_df_covariates_updated.xlsx', index=False)
# logging.info("Updated DataFrame has been saved to filtered_df_covariates_updated.xlsx")


if __name__ == "__main__":
    main()

old code working


In [18]:
# Save the updated DataFrame to files in folder '03_data_ai'
responses.to_excel('03_data_ai/part_3_covariates_done_v2.xlsx', index=False)

# result_df.to_parquet('03_data_ai/part_3_covariates_done_v2.parquet', index=False)



In [19]:
1=2


In [20]:
import os
import requests
import json
import logging
import pandas as pd
import time

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s:%(message)s')

# Configuration Variables
API_KEY = openrouter.api_key 
if not API_KEY:
    logging.error("API_KEY is not set. Please set the OPENROUTER_API_KEY environment variable.")
    exit(1)

SITE_URL = 'https://pd-research.com'       # Your site's URL
APP_NAME = 'PD Research'                   # Your application's name

# Define the models to query with their specific settings
MODELS = {
    "gpt-4": {
        "requires_image": False,
        "payload_modifier": lambda content: content,  # No modification needed
        "max_tokens": 4096,                           # Example max_tokens for GPT-4
        "temperature": 0.5                             # Lower temperature for accuracy
    },
    "anthropic/claude-3.5-sonnet": {
        "requires_image": False,                      # Changed to False as per your requirement
        "payload_modifier": lambda content: content,  # No modification needed
        "max_tokens": 10000,                           # Example max_tokens for Claude 3.5
        "temperature": 0.5                             # Lower temperature for accuracy
    },
    "meta-llama/llama-3-70b-instruct": {
        "requires_image": False,
        "payload_modifier": lambda content: content,  # No modification needed
        "max_tokens": 4096,                           # Example max_tokens for Qwen
        "temperature": 0.5                             # Lower temperature for accuracy
    }
}

# Define colors for each model (Optional: Requires colorama)
try:
    from colorama import Fore, Style
    COLORAMA_AVAILABLE = True
except ImportError:
    COLORAMA_AVAILABLE = False
    logging.warning("Colorama not installed. Responses will not be color-coded.")

MODEL_COLORS = {
    "gpt-4": Fore.BLUE if COLORAMA_AVAILABLE else "",
    "anthropic/claude-3.5-sonnet": Fore.MAGENTA if COLORAMA_AVAILABLE else "",
    "meta-llama/llama-3-70b-instruct": Fore.GREEN if COLORAMA_AVAILABLE else ""
}

def send_request(model_name, prompt):
    """
    Sends a request to the specified model with the given prompt.

    Parameters:
        model_name (str): The name of the model to query.
        prompt (str): The text prompt/question.

    Returns:
        str: The assistant's reply or an error message.
    """
    logging.info(f"Sending request to model: {model_name}")

    # For text-only models, no modification is needed
    content = prompt
    modified_content = MODELS[model_name]["payload_modifier"](content)

    # Construct the payload with per-model settings
    payload = {
        "model": model_name,
        "messages": [
            {
                "role": "user",
                "content": modified_content
            }
        ],
        "top_p": 1,
        "temperature": MODELS[model_name]["temperature"],  # Use per-model temperature
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "repetition_penalty": 1,
        "top_k": 0,
        "max_tokens": MODELS[model_name]["max_tokens"],    # Use per-model max_tokens
    }

    retries = 3
    backoff = 2  # seconds

    for attempt in range(1, retries + 1):
        try:
            # Make the POST request to the OpenRouter API
            response = requests.post(
                url="https://openrouter.ai/api/v1/chat/completions",
                headers={
                    "Authorization": f"Bearer {API_KEY}",
                    "HTTP-Referer": SITE_URL,  # Optional: Include if you want your app featured
                    "X-Title": APP_NAME,        # Optional: Shows your app name on OpenRouter rankings
                    "Content-Type": "application/json"  # Ensure content type is set
                },
                data=json.dumps(payload),
                timeout=60  # Increased timeout for models that may take longer
            )

            logging.info("API request sent. Awaiting response...")

            # Check Content-Type
            content_type = response.headers.get('Content-Type', '')
            logging.debug(f"Response Content-Type: {content_type}")
            if 'application/json' not in content_type:
                logging.error(f"Unexpected Content-Type: {content_type}")
                logging.error(f"Response: {response.text}")
                return "Error: Unexpected Content-Type"

            # Check if the request was successful
            if response.status_code == 200:
                logging.info(f"Received successful response from {model_name}.")
                try:
                    response_data = response.json()
                    # Extract the assistant's reply
                    assistant_reply = response_data['choices'][0]['message']['content'].strip()
                    logging.info(f"Assistant's reply from {model_name}: {assistant_reply}")
                    return assistant_reply
                except (KeyError, IndexError) as e:
                    logging.error(f"Error extracting assistant's reply from {model_name}: {e}")
                    logging.debug(f"Full response from {model_name}: {json.dumps(response_data, indent=4)}")
                    return "Error extracting reply."
            elif response.status_code in [429, 500, 502, 503, 504]:
                # Retry for rate limiting or server errors
                logging.warning(f"Attempt {attempt} failed with status code {response.status_code}. Retrying in {backoff} seconds...")
                time.sleep(backoff)
                backoff *= 2
            else:
                logging.error(f"Request to {model_name} failed with status code {response.status_code}")
                logging.error(f"Response: {response.text}")
                return f"Error: Received status code {response.status_code}"
        except requests.exceptions.RequestException as e:
            logging.error(f"Attempt {attempt} failed with exception: {e}")
            if attempt < retries:
                logging.info(f"Retrying in {backoff} seconds...")
                time.sleep(backoff)
                backoff *= 2
            else:
                logging.error("All retry attempts failed.")
                return f"Error: {e}"

    return "Error: Unable to retrieve response."

def main():
    # Load your DataFrame
    # Replace the following line with how you load your DataFrame
    # For example, if it's a CSV:
    # filtered_df_covariates = pd.read_csv('filtered_df_covariates.csv')

    # For demonstration, let's assume filtered_df_covariates is already loaded
    # Here's a placeholder:
    # filtered_df_covariates = pd.DataFrame(...)  # Your DataFrame loading logic

    # **Important:** Replace the following line with your actual DataFrame loading method
    # For example:
    # filtered_df_covariates = pd.read_csv('your_file.csv')
    # Ensure that 'filtered_df_covariates' is defined before proceeding
    try:
        filtered_df_covariates  # Check if the DataFrame is already loaded
    except NameError:
        logging.error("DataFrame 'filtered_df_covariates' is not defined. Please load it before running the script.")
        exit(1)

    # Define the prompt
    prompt_template = """
    Analyze whether brain stimulation was used in this trial. If so, provide details.

    IMPORTANT: Respond ONLY with a JSON object in the EXACT format below. Do not include any additional text or explanations.

    {
        "brain_stimulation_used": "Yes" or "No",
        "stimulation_details": {
            "primary_type": "e.g., tDCS, TMS, tACS, DBS, etc." or null,
            "is_noninvasive": true or false,
            "primary_target": "Primary brain region or null",
            "secondary_targets": ["List of secondary regions"] or [],
            "stimulation_parameters": {
                "frequency": "e.g., 10Hz" or null,
                "intensity": "e.g., 2mA" or null,
                "duration": "e.g., 20 minutes" or null
            }
        },
        "confidence_level": "High", "Medium", or "Low",
        "relevant_quotes": ["Direct quotes supporting the analysis"]
    }
    """

    # Select the first 5 rows
    first_five = filtered_df_covariates.copy()

    # Define new columns for JSON fields with model suffixes
    # We'll create columns dynamically as we process each model
    # Alternatively, initialize them if you know the models beforehand

    # Initialize the responses list
    responses = []

    # Iterate over the first 5 rows
    for idx, row in first_five.iterrows():
        # Extract relevant information from the row to include in the prompt
        # Customize this based on which columns provide context for the analysis
        # For example, using 'DetailedDescription' or 'FullAnalysis'
        context = row['DetailedDescription'] if 'DetailedDescription' in row else row['BriefSummary']

        # Construct the full prompt by appending context to the prompt template
        prompt = f"{prompt_template}\n\nContext:\n{context}"

        # Send the prompt to each model and store responses
        for model in MODELS.keys():
            response = send_request(model, prompt)
            responses.append({
                "Model": model,
                "Response": response
            })

            # Check if the response is a JSON object
            try:
                response_json = json.loads(response)
            except json.JSONDecodeError:
                logging.error(f"Response from {model} is not valid JSON. Response: {response}")
                response_json = {}

            # Define column suffix based on model name
            suffix = model.replace("/", "_").replace("-", "_")

            # Populate the new columns with the JSON data
            if "brain_stimulation_used" in response_json:
                first_five.at[idx, f"brain_stimulation_used_{suffix}"] = response_json.get("brain_stimulation_used", None)
            else:
                first_five.at[idx, f"brain_stimulation_used_{suffix}"] = None

            if "stimulation_details" in response_json and isinstance(response_json["stimulation_details"], dict):
                stim_details = response_json["stimulation_details"]
                first_five.at[idx, f"stimulation_details_primary_type_{suffix}"] = stim_details.get("primary_type", None)
                first_five.at[idx, f"stimulation_details_is_noninvasive_{suffix}"] = stim_details.get("is_noninvasive", None)
                first_five.at[idx, f"stimulation_details_primary_target_{suffix}"] = stim_details.get("primary_target", None)
                # Assign as JSON string to avoid issues with lists
                secondary_targets = stim_details.get("secondary_targets", [])
                first_five.at[idx, f"stimulation_details_secondary_targets_{suffix}"] = json.dumps(secondary_targets)

                stim_params = stim_details.get("stimulation_parameters", {})
                first_five.at[idx, f"stimulation_details_stimulation_parameters_frequency_{suffix}"] = stim_params.get("frequency", None)
                first_five.at[idx, f"stimulation_details_stimulation_parameters_intensity_{suffix}"] = stim_params.get("intensity", None)
                first_five.at[idx, f"stimulation_details_stimulation_parameters_duration_{suffix}"] = stim_params.get("duration", None)
            else:
                # If 'stimulation_details' is missing or not a dict, set defaults
                first_five.at[idx, f"stimulation_details_primary_type_{suffix}"] = None
                first_five.at[idx, f"stimulation_details_is_noninvasive_{suffix}"] = None
                first_five.at[idx, f"stimulation_details_primary_target_{suffix}"] = None
                first_five.at[idx, f"stimulation_details_secondary_targets_{suffix}"] = json.dumps([])
                first_five.at[idx, f"stimulation_details_stimulation_parameters_frequency_{suffix}"] = None
                first_five.at[idx, f"stimulation_details_stimulation_parameters_intensity_{suffix}"] = None
                first_five.at[idx, f"stimulation_details_stimulation_parameters_duration_{suffix}"] = None

            if "confidence_level" in response_json:
                first_five.at[idx, f"confidence_level_{suffix}"] = response_json.get("confidence_level", None)
            else:
                first_five.at[idx, f"confidence_level_{suffix}"] = None

            if "relevant_quotes" in response_json:
                # Assign as JSON string to avoid issues with lists
                relevant_quotes = response_json.get("relevant_quotes", [])
                first_five.at[idx, f"relevant_quotes_{suffix}"] = json.dumps(relevant_quotes)
            else:
                first_five.at[idx, f"relevant_quotes_{suffix}"] = json.dumps([])

    # Display the updated DataFrame
    print("\n=== Updated LLM Responses ===")
    # Select only the new columns for display
    new_columns_display = []
    for model in MODELS.keys():
        suffix = model.replace("/", "_").replace("-", "_")
        new_columns_display.extend([
            f"brain_stimulation_used_{suffix}",
            f"stimulation_details_primary_type_{suffix}",
            f"stimulation_details_is_noninvasive_{suffix}",
            f"stimulation_details_primary_target_{suffix}",
            f"stimulation_details_secondary_targets_{suffix}",
            f"stimulation_details_stimulation_parameters_frequency_{suffix}",
            f"stimulation_details_stimulation_parameters_intensity_{suffix}",
            f"stimulation_details_stimulation_parameters_duration_{suffix}",
            f"confidence_level_{suffix}",
            f"relevant_quotes_{suffix}"
        ])

    print(first_five[new_columns_display])

    # Optional: Using PrettyTable for better console display
    try:
        from prettytable import PrettyTable
        table = PrettyTable()
        table.field_names = ["Model", "Response"]

        for resp in responses:
            table.add_row([resp["Model"], resp["Response"]])

        print("\n=== LLM Responses (PrettyTable) ===")
        print(table)
    except ImportError:
        logging.warning("PrettyTable not installed. Skipping table formatting.")

    # Optional: Using colorama for color-coded console output
    if COLORAMA_AVAILABLE:
        print("\n=== LLM Responses (Color-Coded) ===")
        for resp in responses:
            color = MODEL_COLORS.get(resp["Model"], Fore.WHITE)
            print(f"{color}{resp['Model']}: {resp['Response']}{Style.RESET_ALL}")
    else:
        logging.info("Colorama not available. Skipping color-coded output.")

    # Optional: Save the updated DataFrame to a file
    # Uncomment the following lines to save as CSV or Excel
    # first_five.to_csv('filtered_df_covariates_updated.csv', index=False)
    # Ensure the new columns are in the DataFrame
    for model in MODELS.keys():
        suffix = model.replace("/", "_").replace("-", "_")
        new_columns = [
            f"brain_stimulation_used_{suffix}",
            f"stimulation_details_primary_type_{suffix}",
            f"stimulation_details_is_noninvasive_{suffix}",
            f"stimulation_details_primary_target_{suffix}",
            f"stimulation_details_secondary_targets_{suffix}",
            f"stimulation_details_stimulation_parameters_frequency_{suffix}",
            f"stimulation_details_stimulation_parameters_intensity_{suffix}",
            f"stimulation_details_stimulation_parameters_duration_{suffix}",
            f"confidence_level_{suffix}",
            f"relevant_quotes_{suffix}"
        ]
        for col in new_columns:
            if col not in first_five.columns:
                first_five[col] = None

    # Ensure the directory '03_data_ai' exists
    os.makedirs('03_data_ai', exist_ok=True)

    # Save the updated DataFrame to files in folder '03_data_ai'
    first_five.to_excel('03_data_ai/part_3_covariates_done.xlsx', index=False)
    first_five.to_parquet('03_data_ai/part_3_covariates_done.parquet', index=False)


    # first_five.to_excel('filtered_df_covariates_updated.xlsx', index=False)
    # logging.info("Updated DataFrame has been saved to filtered_df_covariates_updated.xlsx")

if __name__ == "__main__":
    main()

In [26]:
# # Save the DataFrame to a CSV file
# filtered_df_covariates.to_csv('03_data_ai/part_3_covariates_done.csv', index=False)
# filtered_df_covariates.to_excel('03_data_ai/part_3_covariates_done.xlsx', index=False)
# filtered_df_covariates.to_parquet('03_data_ai/part_3_covariates_done.parquet', index=False)

In [27]:
# Save the DataFrame to an Excel file

In [23]:
1=2


In [24]:
# WORKING - BELOW÷

In [25]:
# import os
# import requests
# import json
# import logging
# import pandas as pd
# import time

# # Configure logging
# # logging.basicConfig(level=logging.INFO, format='%(levelname)s:%(message)s')


# SITE_URL = 'https://pd-research.com'       # Your site's URL
# APP_NAME = 'PD Research'                   # Your application's name

# # Define the models to query with their specific settings
# MODELS = {
#     "gpt-4": {
#         "requires_image": False,
#         "payload_modifier": lambda content: content,  # No modification needed
#         "max_tokens": 4096,                           # Example max_tokens for GPT-4
#         "temperature": 0.5                             # Lower temperature for accuracy
#     },
#     "anthropic/claude-3.5-sonnet": {
#         "requires_image": False,                      # Changed to False as per your requirement
#         "payload_modifier": lambda content: content,  # No modification needed
#         "max_tokens": 10000,                           # Example max_tokens for Claude 3.5
#         "temperature": 0.5                             # Lower temperature for accuracy
#     },
#     "qwen/qwen-2.5-72b-instruct": {
#         "requires_image": False,
#         "payload_modifier": lambda content: content,  # No modification needed
#         "max_tokens": 4096,                           # Example max_tokens for Qwen
#         "temperature": 0.5                             # Lower temperature for accuracy
#     }
# }

# # Define colors for each model (Optional: Requires colorama)
# try:
#     from colorama import Fore, Style
#     COLORAMA_AVAILABLE = True
# except ImportError:
#     COLORAMA_AVAILABLE = False
#     logging.warning("Colorama not installed. Responses will not be color-coded.")

# MODEL_COLORS = {
#     "gpt-4": Fore.BLUE if COLORAMA_AVAILABLE else "",
#     "anthropic/claude-3.5-sonnet": Fore.MAGENTA if COLORAMA_AVAILABLE else "",
#     "qwen/qwen-2.5-72b-instruct": Fore.GREEN if COLORAMA_AVAILABLE else ""
# }

# def send_request(model_name, prompt):
#     """
#     Sends a request to the specified model with the given prompt.
    
#     Parameters:
#         model_name (str): The name of the model to query.
#         prompt (str): The text prompt/question.
        
#     Returns:
#         str: The assistant's reply or an error message.
#     """
#     logging.info(f"Sending request to model: {model_name}")
    
#     # For text-only models, no modification is needed
#     content = prompt
#     modified_content = MODELS[model_name]["payload_modifier"](content)
    
#     # Construct the payload with per-model settings
#     payload = {
#         "model": model_name,
#         "messages": [
#             {
#                 "role": "user",
#                 "content": modified_content
#             }
#         ],
#         "top_p": 1,
#         "temperature": MODELS[model_name]["temperature"],  # Use per-model temperature
#         "frequency_penalty": 0,
#         "presence_penalty": 0,
#         "repetition_penalty": 1,
#         "top_k": 0,
#         "max_tokens": MODELS[model_name]["max_tokens"],    # Use per-model max_tokens
#     }
    
#     retries = 3
#     backoff = 2  # seconds

#     for attempt in range(1, retries + 1):
#         try:
#             # Make the POST request to the OpenRouter API
#             response = requests.post(
#                 url="https://openrouter.ai/api/v1/chat/completions",
#                 headers={
#                     "Authorization": f"Bearer {API_KEY}",
#                     "HTTP-Referer": SITE_URL,  # Optional: Include if you want your app featured
#                     "X-Title": APP_NAME,        # Optional: Shows your app name on OpenRouter rankings
#                     "Content-Type": "application/json"  # Ensure content type is set
#                 },
#                 data=json.dumps(payload),
#                 timeout=60  # Increased timeout for models that may take longer
#             )
            
#             logging.info("API request sent. Awaiting response...")
            
#             # Check Content-Type
#             content_type = response.headers.get('Content-Type', '')
#             logging.debug(f"Response Content-Type: {content_type}")
#             if 'application/json' not in content_type:
#                 logging.error(f"Unexpected Content-Type: {content_type}")
#                 logging.error(f"Response: {response.text}")
#                 return "Error: Unexpected Content-Type"
            
#             # Check if the request was successful
#             if response.status_code == 200:
#                 logging.info(f"Received successful response from {model_name}.")
#                 try:
#                     response_data = response.json()
#                     # Extract the assistant's reply
#                     assistant_reply = response_data['choices'][0]['message']['content'].strip()
#                     logging.info(f"Assistant's reply from {model_name}: {assistant_reply}")
#                     return assistant_reply
#                 except (KeyError, IndexError) as e:
#                     logging.error(f"Error extracting assistant's reply from {model_name}: {e}")
#                     logging.debug(f"Full response from {model_name}: {json.dumps(response_data, indent=4)}")
#                     return "Error extracting reply."
#             elif response.status_code in [429, 500, 502, 503, 504]:
#                 # Retry for rate limiting or server errors
#                 logging.warning(f"Attempt {attempt} failed with status code {response.status_code}. Retrying in {backoff} seconds...")
#                 time.sleep(backoff)
#                 backoff *= 2
#             else:
#                 logging.error(f"Request to {model_name} failed with status code {response.status_code}")
#                 logging.error(f"Response: {response.text}")
#                 return f"Error: Received status code {response.status_code}"
#         except requests.exceptions.RequestException as e:
#             logging.error(f"Attempt {attempt} failed with exception: {e}")
#             if attempt < retries:
#                 logging.info(f"Retrying in {backoff} seconds...")
#                 time.sleep(backoff)
#                 backoff *= 2
#             else:
#                 logging.error("All retry attempts failed.")
#                 return f"Error: {e}"
    
#     return "Error: Unable to retrieve response."

# def main():
#     # Define the prompt (no image URL since all models are text-only)
#     prompt = "What is the average age Parkinson's disease starts?"
    
#     # Prepare a list to hold the responses
#     responses = []
    
#     for model in MODELS.keys():
#         reply = send_request(model, prompt)
#         responses.append({
#             "Model": model,
#             "Response": reply
#         })
    
#     # Create a pandas DataFrame to display the responses
#     df = pd.DataFrame(responses)
    
#     # Debug: Verify the DataFrame structure
#     logging.debug("DataFrame before pivoting:")
#     logging.debug(df)
    
#     # Check if 'Model' and 'Response' columns exist
#     if 'Model' not in df.columns or 'Response' not in df.columns:
#         logging.error("DataFrame missing 'Model' or 'Response' columns.")
#         exit(1)
    
#     # Check for any None or NaN values in 'Model' column
#     if df['Model'].isnull().any():
#         logging.error("Found None or NaN values in 'Model' column.")
#         exit(1)
    
#     # Set 'Model' as the index and transpose the DataFrame
#     df_pivot = df.set_index('Model').transpose()
    
#     # Display the DataFrame
#     print("\n=== LLM Responses ===")
#     print(df_pivot.to_string(index=False))
    
#     # Optional: Using PrettyTable for better console display
#     try:
#         from prettytable import PrettyTable
#         table = PrettyTable()
#         table.field_names = ["Model", "Response"]
        
#         for resp in responses:
#             table.add_row([resp["Model"], resp["Response"]])
        
#         print("\n=== LLM Responses (PrettyTable) ===")
#         print(table)
#     except ImportError:
#         logging.warning("PrettyTable not installed. Skipping table formatting.")
    
#     # Optional: Using colorama for color-coded console output
#     if COLORAMA_AVAILABLE:
#         print("\n=== LLM Responses (Color-Coded) ===")
#         for resp in responses:
#             color = MODEL_COLORS.get(resp["Model"], Fore.WHITE)
#             print(f"{color}{resp['Model']}: {resp['Response']}{Style.RESET_ALL}")
#     else:
#         logging.info("Colorama not available. Skipping color-coded output.")

# if __name__ == "__main__":
#     main()

In [28]:
import requests



resp = requests.post('https://textbelt.com/text', {

  'phone': '9163802941',

  'message': 'LLM Processing Done',

  'key': '138adc496234ca311154757db147f552afa8ba83FfrCKJ36kTJNXq65nlsvvF4Pu',

})

print(resp.json())

In [36]:
import os
import requests
import json
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

# Configuration Variables
API_KEY = openrouter.api_key       # Ensure this environment variable is set
SITE_URL = 'https://pd-research.com'          # Your site's URL
APP_NAME = 'PD Research'                      # Your application's name

def send_test_prompt():
    logging.info("Starting the API request...")
    try:
        # Prepare the payload
        payload = {
                "model": "openai/o1-mini",
                
                "messages": [
                    {"role": "user", "content": "What is the average age Parkinson's disease starts?"}
                ],
                "top_p": 1,
                "temperature": 1,
                "frequency_penalty": 0,
                "presence_penalty": 0,
                "repetition_penalty": 1,
                "top_k": 0,
                }

        # Make the POST request to the OpenRouter API
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {API_KEY}",
                "HTTP-Referer": SITE_URL,  # Optional: Include if you want your app featured
                "X-Title": APP_NAME,        # Optional: Shows your app name on OpenRouter rankings
            },
            data=json.dumps(payload)
        )

        logging.info("API request sent. Awaiting response...")

        # Check if the request was successful
        if response.status_code == 200:
            logging.info("Received successful response from the API.")
            response_data = response.json()

            # Print the full JSON response for debugging
            logging.debug("Full Response JSON:")
            logging.debug(json.dumps(response_data, indent=4))

            # Extract the assistant's reply
            try:
                assistant_reply = response_data['choices'][0]['message']['content'].strip()
                logging.info(f"Assistant's reply: {assistant_reply}")
            except (KeyError, IndexError) as e:
                logging.error("Error extracting assistant's reply:", exc_info=True)
                logging.debug("Full response:", json.dumps(response_data, indent=4))
        else:
            logging.error(f"Request failed with status code {response.status_code}")
            logging.error(f"Response: {response.text}")

    except requests.exceptions.RequestException as e:
        logging.error("An error occurred while making the request:", exc_info=True)

# --- Execute the Test Prompt ---
if __name__ == "__main__":
    send_test_prompt()

In [41]:
ß

In [47]:
1=2


In [53]:
import pandas as pd
import openai

# Set your OpenAI API key
# openai.api_key = 'YOUR_OPENAI_API_KEY'  # Replace with your actual API key

# Function to query OpenAI for each row
def get_brain_area(row):
    # Prepare the prompt with the relevant data from the row
    prompt = f"""Given the following study data:

{row.to_json()}

What area of the brain are they looking for?"""

    try:
        response = openai.chat.completions.create(
            model='gpt-3.5-turbo',  # Use 'gpt-4' if you have access
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=100,
            temperature=0.5,
        )
        # Extract the assistant's reply
        answer = response.choices[0].message.content.strip()
        # Print the response
        print(f"Row {row.name} response: {answer}")
        return answer
    except Exception as e:
        print(f"Error processing row {row.name}: {e}")
        return None

# Ensure that 'filtered_df_covariates' is defined with your DataFrame
# For example:
# filtered_df_covariates = pd.read_csv('your_data.csv')

# Apply the function to each row and save the result in a new column
filtered_df_covariates['BrainArea'] = filtered_df_covariates.apply(get_brain_area, axis=1)

In [54]:
import pandas as pd
import openai

# Set your OpenAI API key
# openai.api_key = 'YOUR_OPENAI_API_KEY'  # Replace with your actual API key

# Function to query OpenAI for each row
def get_brain_area(row):
    # Prepare the prompt with the relevant data from the row
    prompt = f"""Given the following study data:

{row.to_json()}

What area of the brain are they looking for?"""

    try:
        response = openai.chat.completions.create(
            model='gpt-3.5-turbo',  # Use 'gpt-4' if you have access
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=100,
            temperature=0.5,
        )
        # Extract the assistant's reply
        answer = response.choices[0].message.content.strip()
        # Print the response
        print(f"Row {row.name} response: {answer}")
        return answer
    except Exception as e:
        print(f"Error processing row {row.name}: {e}")
        return None

# Ensure that 'filtered_df_covariates' is defined with your DataFrame
# For example:
# filtered_df_covariates = pd.read_csv('your_data.csv')

# Apply the function to each row and save the result in a new column
filtered_df_covariates['BrainArea'] = filtered_df_covariates.apply(get_brain_area, axis=1)

In [56]:
# import pandas as pd
# import openai

# # Set your OpenAI API key

# # Function to query OpenAI for each row
# def get_brain_area(row):
#     # Prepare the prompt with the relevant data from the row
#     prompt = f"""Given the following study data:

# {row.to_json()}

# What area of the brain are they looking for?"""

#     try:
#         response = openai.chat.completions.create(
#             model='gpt-3.5-turbo',  # Use 'gpt-4' if you have access
#             messages=[
#                 {"role": "user", "content": prompt}
#             ],
#             max_tokens=100,
#             temperature=0.5,
#         )
#         # Extract the assistant's reply
#         answer = response.choices[0].message.content.strip()
#         return answer
#     except Exception as e:
#         print(f"Error processing row {row.name}: {e}")
#         return None

# # Ensure that 'filtered_df_covariates' is defined with your DataFrame
# # For example:
# # filtered_df_covariates = pd.read_csv('your_data.csv')

# # Apply the function to each row and save the result in a new column
# filtered_df_covariates['BrainArea'] = filtered_df_covariates.apply(get_brain_area, axis=1)

In [66]:
import openai
print(openai.__version__)


In [71]:
def _call_local_llama(self, prompt: str) -> str:
    """Call local Llama instance using Ollama API"""
    try:
        # Construct the API endpoint
        api_endpoint = f"{self.local_url}generate"


        # Prepare the request payload
        payload = {
            "model": self.local_model,
            "prompt": prompt,
            "options": {
                "temperature": 0.1
            }
        }

        # Send the POST request
        response = requests.post(api_endpoint, json=payload)

        if response.status_code == 200:
            # Extract the generated text from the response
            result = response.json()
            if 'generated_text' in result:
                return result['generated_text']
            else:
                print(f"Unexpected response structure: {result}")
                return ''
        else:
            raise Exception(f"Local LLM error: Status {response.status_code} - {response.text}")

    except requests.exceptions.RequestException as e:
        raise Exception(f"Network error calling Ollama: {str(e)}")
    except json.JSONDecodeError as e:
        raise Exception(f"Error parsing Ollama response: {str(e)}")
    except Exception as e:
        raise Exception(f"Error calling local Llama: {str(e)}")

In [73]:
import openai

# Your data
data_json = '{"study": "example study data"}'
prompt = f"Given the following study data, provide detailed information about the type of study and the brain area involved:\n\n{data_json}"

# Make the API request to OpenAI
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=1500,
    n=1,
    stop=None,
    temperature=0.5,
)

# Print the response
print(response['choices'][0]['message']['content'].strip())